In [1]:
data_var = '2024-02-12'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11522,2024-02-12,Brasil Nbb,20:00,Pinheiros,Franca,4.41,1.19,164.5,1.85,1.85,9.5,1.94,1.75,C2daQ2b3,0.226757,0.840336,0.540541,0.540541,0.067094,0.8,1.0,NaN,NaN,146.902,56.238026,0.382827,131.028,32.550102,0.248421,157.876,147.096,148.68,148.80,0.0,0.0,0.0,0.0,0.813173,0.000000,0.072819,-1.88,-0.376,-9.069149,0.605064,0.6,-0.005064,0.57,0.114,1.666667,0.734929,0.8,0.065071
11523,2024-02-12,Eua Nba,21:00,Charlotte Hornets,Indiana Pacers,5.68,1.16,237.5,1.82,2.06,10.5,2.02,1.92,A5KfmwH4,0.176056,0.862069,0.549451,0.485437,0.038125,0.6,0.4,NaN,NaN,459.554,181.564435,0.395088,203.282,113.833799,0.559980,378.458,292.632,271.40,140.30,0.0,0.0,0.0,0.0,0.934539,0.087477,0.035894,-2.64,-0.528,-8.863636,0.325743,0.2,-0.125743,-0.06,-0.012,-13.333333,0.411814,0.5,0.088186
11524,2024-02-12,Eua Nba,21:00,Cleveland Cavaliers,Philadelphia 76ers,1.27,4.07,227.5,1.89,2.03,-10.5,2.04,1.88,ldJbncWA,0.787402,0.245700,0.529101,0.492611,0.033102,0.4,0.4,NaN,NaN,175.580,47.121406,0.268376,244.778,67.204307,0.274552,229.724,241.598,195.84,189.21,0.0,0.0,0.0,0.0,0.741535,0.050508,0.057723,4.37,0.874,0.308924,0.604552,0.8,0.195448,-0.69,-0.138,-22.246377,0.560435,0.4,-0.160435
11525,2024-02-12,Eua Nba,21:30,Atlanta Hawks,Chicago Bulls,1.61,2.41,235.5,1.90,2.02,-3.5,1.99,1.96,IVN2oHoH,0.621118,0.414938,0.526316,0.495050,0.036056,0.8,1.0,NaN,NaN,299.474,149.137976,0.498000,269.586,86.347481,0.320297,250.032,273.302,186.66,290.52,0.0,0.0,0.0,0.0,0.281436,0.043292,0.010741,1.94,0.388,1.572165,0.591285,0.6,0.008715,-1.89,-0.378,-3.730159,0.405872,0.4,-0.005872
11526,2024-02-12,Eua Nba,21:30,Toronto Raptors,San Antonio Spurs,1.31,3.68,231.5,1.83,2.04,-7.5,2.03,1.91,U9w4ozbU,0.763359,0.271739,0.546448,0.490196,0.035098,0.4,0.6,NaN,NaN,345.370,166.630136,0.482468,394.222,92.186186,0.233843,256.506,527.938,361.95,277.07,1.0,0.0,0.0,0.0,0.671681,0.076740,0.043072,-3.34,-0.668,-0.464072,0.477398,0.2,-0.277398,-5.00,-1.000,-2.680000,0.290624,0.1,-0.190624
11527,2024-02-12,Eua Nba,22:00,Houston Rockets,New York Knicks,2.78,1.48,219.5,1.88,2.05,-7.5,1.90,1.90,dMM6py1N,0.359712,0.675676,0.531915,0.487805,0.035388,0.6,0.0,NaN,NaN,262.170,63.979095,0.244037,170.866,31.934420,0.186897,259.082,207.742,187.65,155.94,0.0,0.0,0.0,0.0,0.431568,0.061175,0.000000,-1.85,-0.370,-4.810811,0.547165,0.5,-0.047165,3.84,0.768,0.625000,0.541662,0.7,0.158338
11528,2024-02-12,Eua Nba,22:00,Memphis Grizzlies,New Orleans Pelicans,4.07,1.27,221.5,1.85,2.07,8.5,1.99,1.95,voCBqeGT,0.245700,0.787402,0.540541,0.483092,0.033102,0.4,0.2,NaN,NaN,421.222,225.120100,0.534445,201.084,97.106240,0.482914,340.382,205.200,316.80,121.83,1.0,0.0,0.0,0.0,0.741535,0.079369,0.014357,1.53,0.306,10.032680,0.445217,0.4,-0.045217,2.42,0.484,0.557851,0.510810,0.6,0.089190
11529,2024-02-12,Eua Nba,22:00,Milwaukee Bucks,Denver Nuggets,2.04,1.91,232.5,1.84,2.10,1.5,1.91,2.04,G63wuFOp,0.490196,0.523560,0.543478,0.476190,0.013756,0.8,0.4,NaN,NaN,194.722,43.342264,0.222585,174.980,47.351693,0.270612,171.530,192.356,174.30,233.20,0.0,0.0,0.0,0.0,0.046544,0.093324,0.046544,-0.92,-0.184,-5.652174,0.730959,0.7,-0.030959,-1.71,-0.342,-2.660819,0.581398,0.5,-0.081398
11530,2024-02-12,Eua Nba,22:30,Dallas Mavericks,Washington Wizards,1.13,6.63,246.5,1.89,2.03,-14.5,2.00,1.94,U9iL3b1o,0.884956,0.150830,0.529101,0.492611,0.035785,0.8,0.2,NaN,NaN,238.622,51.104373,0.214165,530.506,240.796358,0.453899,259.806,512.276,281.78,895.26,0.0,1.0,0.0,0.0,1.002342,0.

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Charlotte Hornets,Indiana Pacers,237.5,1.82,1.0000,Over
1,21:00,Eua Nba,Cleveland Cavaliers,Philadelphia 76ers,227.5,1.89,1.0000,Over
2,21:30,Eua Nba,Atlanta Hawks,Chicago Bulls,235.5,1.90,1.0000,Over
3,21:30,Eua Nba,Toronto Raptors,San Antonio Spurs,231.5,1.83,1.0000,Over
4,22:00,Eua Nba,Houston Rockets,New York Knicks,219.5,1.88,0.9996,Over
5,22:00,Eua Nba,Memphis Grizzlies,New Orleans Pelicans,221.5,1.85,1.0000,Over
6,22:00,Eua Nba,Milwaukee Bucks,Denver Nuggets,232.5,1.84,1.0000,Over
7,22:30,Eua Nba,Dallas Mavericks,Washington Wizards,246.5,1.89,1.0000,Over
8,23:00,Eua Nba,Utah Jazz,Golden State Warriors,242.5,1.89,1.0000,Over
9,11:00,Rússia Liga Vtb United,Astana,Saratov,160.5,1.86,0.9989,Over
